# **Install Requirements**

In [ ]:
%pip install huggingface_hub
from huggingface_hub import login
login("Put_your_Token_here")
!sudo apt-get install tesseract-ocr -y
!sudo apt-get install tesseract-ocr-ara -y
!sudo pip install pytesseract
%pip install gradio
%pip install llama-stack
%pip install gradio pytesseract opencv-python PyMuPDF python-docx openpyxl python-pptx transformers
%pip install torch tensorflow
%pip install accelerate
%pip install SpeechRecognition pydub

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1

In [ ]:
from requests import get
import os
def download_url(url, filename):
    if not os.path.exists(filename):
        with open(filename, 'wb') as f:
            response = get(url)
            f.write(response.content)
        print(f"Downloaded {filename}")

In [ ]:
download_url("https://github.com/tesseract-ocr/tessdata/raw/master/ara.traineddata", "ara.traineddata")

Downloaded ara.traineddata


In [ ]:
import cv2
import gradio as gr
import pytesseract
import fitz  # PyMuPDF
from transformers import pipeline
import numpy as np
import torch
import docx  # For Word files
import openpyxl  # For Excel files
from pptx import Presentation  # For PowerPoint files
import speech_recognition as sr
from pydub import AudioSegment

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

# **Summarization Model**

In [ ]:
class SummarizerExpert:
    def __init__(self, model_name: str = "meta-llama/Llama-3.2-1B-Instruct", max_length: int = 5000, device: str = "cuda:0"):
        try:
            self.pipe = pipeline("text-generation", model=model_name, max_length=max_length, device=device)
        except Exception as e:
            raise Exception(f"Error loading the summarization model: {str(e)}")

    def extract_text(self, image, lang='ara'):
        """Extract text from an image using OCR."""
        try:
            text = pytesseract.image_to_string(image, lang=lang, config="--psm 6")
            return text
        except Exception as e:
            return f"Error in OCR extraction: {str(e)}"

    def extract_text_from_pdf(self, pdf_path, lang='ara'):
        """Extract text and images from a PDF file."""
        extracted_text = ""
        images = []

        try:
            with fitz.open(pdf_path) as doc:
                for page in doc:
                    extracted_text += page.get_text() + "\n"  # Extract text
                    image_list = page.get_images(full=True)
                    for img_index, img in enumerate(image_list):
                        xref = img[0]
                        base_image = doc.extract_image(xref)
                        image_bytes = base_image["image"]
                        img_cv = cv2.imdecode(np.frombuffer(image_bytes, np.uint8), cv2.IMREAD_COLOR)
                        images.append(img_cv)
        except Exception as e:
            return f"Error in PDF extraction: {str(e)}", []

        return extracted_text.strip(), images

    def extract_text_from_word(self, word_path):
        """Extract text from a Word document."""
        extracted_text = ""
        try:
            doc = docx.Document(word_path)
            for para in doc.paragraphs:
                extracted_text += para.text + "\n"
        except Exception as e:
            return f"Error in Word extraction: {str(e)}"
        return extracted_text.strip()

    def extract_text_from_excel(self, excel_path):
        """Extract text from an Excel file."""
        extracted_text = ""
        try:
            wb = openpyxl.load_workbook(excel_path)
            for sheet in wb.sheetnames:
                ws = wb[sheet]
                for row in ws.iter_rows(values_only=True):
                    extracted_text += ' '.join([str(cell) for cell in row if cell is not None]) + "\n"
        except Exception as e:
            return f"Error in Excel extraction: {str(e)}"
        return extracted_text.strip()

    def extract_text_from_powerpoint(self, ppt_path):
        """Extract text from a PowerPoint presentation."""
        extracted_text = ""
        try:
            prs = Presentation(ppt_path)
            for slide in prs.slides:
                for shape in slide.shapes:
                    if hasattr(shape, "text"):
                        extracted_text += shape.text + "\n"
        except Exception as e:
            return f"Error in PowerPoint extraction: {str(e)}"
        return extracted_text.strip()

    def generate_messages(self, text: str, language: str = "arabic",num_words:int=150):
        """Generate messages for summarization."""
        if language.lower() == "arabic":
            messages = [
                {"role": "system",
                 "content": "You are a professional summarization expert with advanced proficiency in the Arabic language. Your task is to create a concise and human-like summary that captures the key points of the given text. The summary should be clear, coherent, and convey the main ideas accurately without adding unnecessary details."},
                {"role": "user",
                 "content": f"Here is the text to summarize in Arabic: {text}. Now provide a summary in Arabic."},
                {"role":"assistant",
                 "content":f"under {num_words} words and in bullet points"}
            ]
        elif language.lower() == "english":
            messages = [
                {"role": "system",
                 "content": "You are a professional summarization expert with advanced proficiency in the English language. Your task is to create a concise and human-like summary that captures the key points of the given text. The summary should be clear, coherent, and convey the main ideas accurately without adding unnecessary details."},
                {"role": "user",
                 "content": f"Here is the text to summarize in English: {text}. Now provide a summary in English in {num_words} words"},
                {"role":"assistant",
                 "content":f"under {num_words} words and in bullet points"}
            ]
        return messages

    def summarize_text(self, text: str, language: str = "arabic", num_words: int = 150):
        """Summarize the given text."""
        try:
            messages = self.generate_messages(text, language,num_words=num_words)
            result = self.pipe(messages)
            return result[0]["generated_text"][3]["content"]
        except Exception as e:
            return f"Error in summarization: {str(e)}"

    def process_image(self, image, lang='eng', summary_lang='english', num_words=150):
        """Process the uploaded image, extract text, and summarize it."""
        image_cv = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        text = self.extract_text(image_cv, lang=lang)

        if "Error" in text:
            return text, "No summary due to OCR error."

        summary = self.summarize_text(text, language=summary_lang, num_words=num_words)

        if "Error" in summary:
            return text, summary

        return text, summary

    def process_text(self, text, summary_lang='english', num_words=150):
        """Process the provided text and summarize it."""
        summary = self.summarize_text(text, language=summary_lang, num_words=num_words)
        return text, summary

# **Question Generation Model**

In [ ]:
class QuestionGeneration:
    def __init__(self, model_name: str = "meta-llama/Llama-3.2-1B-Instruct", max_length: int = 5000, device: str = "cuda:0"):
        self.pipe = pipeline("text-generation", model=model_name, max_length=max_length, device=device)

    def generate_messages(self, text: str, language: str = "arabic", num_of_questions: int = 5):
        if language.lower() == "arabic":
            messages = [
                {"role": "system",
                 "content": "You are an expert question generator with advanced proficiency in the Arabic language. Your task is to generate relevant and meaningful questions based on the given text. The questions should test the comprehension of the key ideas from the text."},
                {"role": "user",
                 "content": f"""Now generate {num_of_questions} questions in Arabic that are labeled from 1 to {num_of_questions} for that text: {text} Please format your response as follows:
                1. السؤال: [The generated question]
                  الاجابة: [The answer based on the paragraph]

                2. السؤال: [Another unique question]
                  الاجابة: [The answer based on the paragraph]

                Continue this format for all questions."""},
                {"role":"assistant",
                 "content":"In Bullet points"}
            ]
        elif language.lower() == "english":
            messages = [
                {"role": "system",
                 "content": "You are an expert question generator with advanced proficiency in the English language. Your task is to generate relevant and meaningful questions based on the given text. The questions should test the comprehension of the key ideas from the text."},
                {"role": "user",
                 "content": f"""Now generate {num_of_questions} questions in English that are labeled from 1 to {num_of_questions} for that text: {text} Please format your response as follows:
              1. Question: [The generated question]
                Answer: [The answer based on the paragraph]

              2. Question: [Another unique question]
                Answer: [The answer based on the paragraph]

              Continue this format for all questions."""},
                 {"role":"assistant",
                 "content":"In Bullet points"}
            ]
        return messages

    def generate_questions(self, text: str, language: str = "arabic", num_of_questions: int = 5):
        messages = self.generate_messages(text, language, num_of_questions)
        return self.pipe(messages)[0]["generated_text"][3]["content"]


# **Speech To Text Model**

In [ ]:
class SpeechRecognizer:
    def __init__(self, language='en-US'):
        """Initialize the speech recognizer with a specific language."""
        self.recognizer = sr.Recognizer()
        self.language = language

    def convert_audio_format(self, audio_file):
        """Convert audio file to WAV format if needed (e.g., from MP3)."""
        if audio_file.endswith('.mp3'):
            audio = AudioSegment.from_mp3(audio_file)
            audio_file = 'converted.wav'
            audio.export(audio_file, format='wav')
        return audio_file

    def transcribe_audio(self, audio_file):
        """Transcribe the audio file using Google Web Speech API."""
        # Convert audio to WAV format if necessary
        audio_file = self.convert_audio_format(audio_file)

        # Load the audio file
        with sr.AudioFile(audio_file) as source:
            audio_data = self.recognizer.record(source)  # Read the entire audio file

        # Recognize speech
        try:
            text = self.recognizer.recognize_google(audio_data, language=self.language)
            return text
        except sr.UnknownValueError:
            return "Sorry, I could not understand the audio."
        except sr.RequestError as e:
            return f"Could not request results from Google Speech Recognition service; {e}"

# **Gradio Interface**

In [ ]:
import gradio as gr

def create_interface():
    summarizer = SummarizerExpert()
    question_generator = QuestionGeneration()


    def process_input(text_input,file, lang, task_type, summary_lang='english', num_words=150, num_questions=5):
        extracted_text = "Please provide text or a file"
        summary = ""
        questions = ""

        if text_input:
          extracted_text=text_input
        if file:
          if file.name.endswith(".pdf"):
            extracted_text, images = summarizer.extract_text_from_pdf(file.name, lang=lang)
          elif file.name.endswith(".docx"):
            extracted_text = summarizer.extract_text_from_word(file.name)
          elif file.name.endswith(".xlsx"):
            extracted_text = summarizer.extract_text_from_excel(file.name)
          elif file.name.endswith(".pptx"):
            extracted_text = summarizer.extract_text_from_powerpoint(file.name)
          elif file.name.endswith((".png", ".jpg", ".jpeg")):  # Check for image formats
            image = cv2.imread(file.name)
            extracted_text, _ = summarizer.process_image(image, lang=lang, summary_lang=summary_lang, num_words=num_words)
          elif file.name.endswith((".mp3", ".wav")):  # Check for audio formats
            lang= "en-US" if lang == "eng" else "ar-EG"
            recognizer = SpeechRecognizer(language=lang)
            extracted_text = recognizer.transcribe_audio(file.name)

        if ("Summarization" in task_type) and (text_input or file):
          summary = summarizer.summarize_text(extracted_text, language=summary_lang, num_words=num_words)

        if ("Question Generation" in task_type) and (text_input or file):
          questions = question_generator.generate_questions(extracted_text, language=summary_lang, num_of_questions=num_questions)

        return extracted_text, summary, questions

    iface = gr.Interface(
        fn=process_input,
        inputs=[
            gr.Textbox(label="Enter text"),
            gr.File(label="Upload your document (PDF/Image/Audio/Excel/Word/PowerPoint/Text)"),
            gr.Radio(["ara", "eng"], label="OCR Language"),
            gr.CheckboxGroup(choices=["Summarization", "Question Generation"], value=["Summarization"], label="Select Task(s)"),
            gr.Radio(["arabic", "english"], label="Summarization Language"),
            gr.Number(label="Summary Length (Number of Words)", value=150),
            gr.Number(label="Number of Questions (if generating)", value=5)
        ],
        outputs=[
            gr.Textbox(label="Extracted Text"),
            gr.Textbox(label="Summary"),
            gr.Textbox(label="Generated Questions")
        ],
        title="Document Analyzer with Summarization & Question Generation",
        description="Upload your document (PDF/Image/Audio/Excel/Word/PowerPoint/Text) and select the tasks you want to perform: text extraction, summarization, or question generation."
    )

    iface.launch(debug=True, share=True)


In [ ]:
create_interface()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://f13b786d52d16a6e89.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for